In [2]:
# %load_ext autoreload
# %autoreload 2

# Tasks registering

In [3]:
@TI.add_kwargs(**{'bs': 32})
@TI.register_fl_task(model='a', data_loader='b', device=None, optimizer=None)
def fl_task(a, b=1, **kwargs):
    print("I am in the body of the task")
    print(a, b, '\n')

In [4]:
TI.task_registry

{'fl_task': <function openfl.interface.python_api.experiment.TaskInterface.register_fl_task.<locals>.decorator_with_args.<locals>.wrapper_decorator(**task_keywords)>}

In [5]:
TI.task_contract

{'fl_task': {'model': 'a',
  'data_loader': 'b',
  'device': None,
  'optimizer': None}}

In [6]:
TI.task_settings

defaultdict(dict, {'fl_task': {'bs': 32}})

# Importing sessions

In [19]:
# dir()

In [18]:
import dill
import sys

In [2]:
sys.path.insert(0, './Kvasir_UNET_workspace/')
# dill.load_session('./Kvasir_UNET_workspace/kvasir_session')

## Import Interface objects

In [16]:
import numpy as np
TI.task_registry['test_task'](np_array=np.arange(10))

tensor([ 2.5016, -2.7028, -1.5531, -1.1858, -3.3686], grad_fn=<AddBackward0>)

In [11]:
import torch
a = torch.rand(1,3,128,128)
model_unet.forward(a)

tensor([[[[0.4349, 0.5119, 0.5558,  ..., 0.4346, 0.4373, 0.5033],
          [0.5192, 0.4408, 0.3518,  ..., 0.5782, 0.4258, 0.4165],
          [0.5452, 0.5024, 0.5299,  ..., 0.4470, 0.4950, 0.4423],
          ...,
          [0.5653, 0.4639, 0.5049,  ..., 0.5477, 0.3774, 0.4458],
          [0.4992, 0.5300, 0.4576,  ..., 0.5732, 0.4905, 0.5270],
          [0.5308, 0.3587, 0.5980,  ..., 0.5927, 0.5575, 0.4409]]]],
       grad_fn=<SigmoidBackward>)

In [37]:
with open('./dataloader.pkl', 'rb') as f:
    fed_dataset = dill.load(f)
    
%cd Kvasir_UNET_workspace/
fed_dataset._delayed_init()


loader = fed_dataset.get_train_loader()
# 

# %cd ..

[Errno 2] No such file or directory: 'Kvasir_UNET_workspace/'
/home/davidyuk/openfl/Plugin_experiments/Kvasir_UNET_workspace


In [38]:
i_l = iter(loader)

In [40]:
%cd Kvasir_UNET_workspace/
imgs, targets = next(i_l)
# %cd ..
imgs.shape

[Errno 2] No such file or directory: 'Kvasir_UNET_workspace/'
/home/davidyuk/openfl/Plugin_experiments/Kvasir_UNET_workspace


torch.Size([8, 3, 332, 332])

In [29]:
pwd

'/home/davidyuk/openfl/Plugin_experiments/Kvasir_UNET_workspace'

# Importing task registry

In [13]:
with open('./Kvasir_UNET_workspace/tasks.pkl', 'rb') as f:
    TI = dill.load(f)

In [15]:
# import torch
# import torch.nn as nn
result = TI.task_registry['test_task'](np_array = list(range(10)))
result

tensor([ 0.6164, -2.1762,  5.4337,  3.4153, -3.9056], grad_fn=<AddBackward0>)

# Importing pytorch model

In [11]:
# with open('./Kvasir_UNET_workspace/model_cls.pkl', 'rb') as f:
#     cls_model = dill.load(f)

with open('./Kvasir_UNET_workspace/model.pkl', 'rb') as f:
    model_interface = dill.load(f)

In [12]:
import torch
a = torch.rand(1,3,128,128)
model_interface.model.forward(a)

tensor([[[[0.5444, 0.4673, 0.4432,  ..., 0.4121, 0.5059, 0.5755],
          [0.5219, 0.6127, 0.4329,  ..., 0.5197, 0.5235, 0.6089],
          [0.5560, 0.4827, 0.5650,  ..., 0.4314, 0.4856, 0.5722],
          ...,
          [0.5542, 0.5896, 0.4729,  ..., 0.5826, 0.5991, 0.6171],
          [0.6715, 0.4237, 0.6175,  ..., 0.5654, 0.5176, 0.6134],
          [0.5557, 0.4961, 0.5315,  ..., 0.4987, 0.6574, 0.4839]]]],
       grad_fn=<SigmoidBackward>)

In [10]:
model_interface.model

model.UNet

In [1]:
from openfl.federated import Plan

/home/davidyuk/.virtualenvs/openfl/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
from pathlib import Path
plan = Plan.Parse(Path('../openfl-workspace/torch_unet_kvasir/plan/plan.yaml'), resolve=False)
plan.config
# plan.Load(Path('../../openfl-workspace/torch_cnn_mnist/plan/plan.yaml'))

{'aggregator': {'defaults': 'plan/defaults/tasks_torch.yaml',
  'template': 'code.fed_unet_runner.PyTorchFederatedUnet',
  'settings': {'init_state_path': 'save/torch_unet_kvasir_init.pbuf',
   'last_state_path': 'save/torch_unet_kvasir_last.pbuf',
   'best_state_path': 'save/torch_unet_kvasir_best.pbuf',
   'rounds_to_train': 40,
   'delta_updates': False,
   'opt_treatment': 'RESET',
   'batch_size': 4,
   'percent_train': 0.8,
   'collaborator_count': 2,
   'data_group_name': 'kvasir',
   'epochs_per_round': 1.0,
   'polling_interval': 4,
   'n_channels': 3,
   'n_classes': 1},
  'aggregated_model_validation': {'function': 'validate',
   'kwargs': {'apply': 'global', 'metrics': ['dice_coef']}},
  'locally_tuned_model_validation': {'function': 'validate',
   'kwargs': {'apply': 'local', 'metrics': ['dice_coef']}},
  'train': {'function': 'train',
   'kwargs': {'batch_size': 32,
    'epochs': 1,
    'metrics': ['loss'],
    'num_batches': 1}}},
 'collaborator': {'defaults': 'plan/defa

In [10]:
plan.config['aggregator']['settings']['init_state_path']

'save/tf_cnn_histology_init.pbuf'

In [8]:
from importlib.resources import Path, path

In [5]:
# with open_text('openfl.interface', 'plan.yaml') as f:
safe_load(read_text('openfl.interface.interactive_api', 'plan.yaml'))

{'aggregator': {'defaults': 'plan/defaults/aggregator.yaml',
  'template': 'openfl.component.Aggregator',
  'settings': {'init_state_path': 'save/init.pbuf',
   'best_state_path': 'save/best.pbuf',
   'last_state_path': 'save/last.pbuf',
   'rounds_to_train': 10}},
 'collaborator': {'defaults': 'plan/defaults/collaborator.yaml',
  'template': 'openfl.component.Collaborator',
  'settings': {'delta_updates': False, 'opt_treatment': 'RESET'}},
 'data_loader': {'defaults': 'plan/defaults/data_loader.yaml'},
 'task_runner': {'defaults': 'plan/defaults/task_runner.yaml'},
 'network': {'defaults': 'plan/defaults/network.yaml'},
 'assigner': {'defaults': 'plan/defaults/assigner.yaml'},
 'tasks': {'defaults': 'plan/defaults/tasks_fast_estimator.yaml'}}

In [3]:
from yaml import safe_load

In [10]:
with path('openfl.interface.interactive_api', 'plan.yaml') as p:
    print(p)

/home/davidyuk/openfl/openfl/interface/interactive_api/plan.yaml


In [15]:
from importlib import resources
from openfl.federated import Plan
with resources.path('openfl.interface.interactive_api', 'plan.yaml') as plan_path:
    plan = Plan.Parse(Path(plan_path), resolve=False)
plan.config

{'aggregator': {'defaults': 'plan/defaults/tasks_fast_estimator.yaml',
  'template': 'openfl.component.Collaborator',
  'settings': {'init_state_path': 'save/init.pbuf',
   'best_state_path': 'save/best.pbuf',
   'last_state_path': 'save/last.pbuf',
   'rounds_to_train': 10,
   'delta_updates': False,
   'opt_treatment': 'RESET'}},
 'collaborator': {'defaults': 'plan/defaults/tasks_fast_estimator.yaml',
  'template': 'openfl.component.Collaborator',
  'settings': {'init_state_path': 'save/init.pbuf',
   'best_state_path': 'save/best.pbuf',
   'last_state_path': 'save/last.pbuf',
   'rounds_to_train': 10,
   'delta_updates': False,
   'opt_treatment': 'RESET'}},
 'data_loader': {'defaults': 'plan/defaults/tasks_fast_estimator.yaml',
  'template': 'openfl.component.Collaborator',
  'settings': {'init_state_path': 'save/init.pbuf',
   'best_state_path': 'save/best.pbuf',
   'last_state_path': 'save/last.pbuf',
   'rounds_to_train': 10,
   'delta_updates': False,
   'opt_treatment': 'RESET